# LIBRARY

In [1]:
# Data Preprocessing
import numpy as np
import pandas as pd
import json

# Data Visualization
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns

# Data Modeling
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import xgboost
import lightgbm

# Text Sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Others
import os
from datetime import datetime
import math
import requests
import re
from pathlib import Path
import pycountry

# Excel
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.chart import LineChart, BarChart, Reference, Series


# Warning Removal
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

In [2]:
now = datetime.now()
current_date = math.floor(datetime.timestamp(now))

one_day_to_second = 60*60*24
one_month_to_second = 60*60*24*30
three_month_to_second = 60*60*24*92

last_1_month = current_date - one_month_to_second
last_3_month = current_date - three_month_to_second

# 1. FACEBOOK

## 1.0 Authentication

In [25]:
# Opening JSON file
f = open('authen.json',)

# returns JSON object as a dictionary
authen = json.load(f)
authen

{'fb_api_vers': 'v10.0',
 'fb_token': 'EAAEZAQPIck2UBADLP14dTUGUaCZALq06VdpOfn6symbhbpa64ZBV10JQxBK59qTTQh36Tv2ZC1TyCqUwN5uaA9ZCwRaQfpBeTV10fDpofyc8BWAfMtg705eM0IlUcQ6u0GrHtt1ajHvujUJoJn525ZCpGJzRNuziDFRgOtmB1xqSNfgu0PK2I4',
 'fb_page_id': '680953308635161',
 'ig_token': 'EAAEZAQPIck2UBAGGCRcT4NFIMbKYMxVL56uAQIV4KgnVVPjWWIjaAiPobz7ORHVzKr5ZBy1W1eF6dkTjzrV7mIdTmdRpJO8AT3EDO7ZAxdZAvcHYuknWVdlMUvK3MAf9K1BxX4OH9CtUv5bZAlaQDRPZBVmRwRTYdqipZACUq0QXAZDZD'}

In [26]:
fb_api_vers = authen['fb_api_vers']
long_access_token = authen['fb_token']
page_id = authen['fb_page_id']

## 1.1 Page Performance
### 1.1.1 Page like, Page View, Page Post Engagement, Page Video View, Page People Reached

In [27]:
def extract_metric_data(metric_name):
    now = datetime.now()
    current_date = math.floor(datetime.timestamp(now))
    three_month_to_second = 60*60*24*92
    one_day_to_second = 60*60*24
    last_3_month = current_date - three_month_to_second

    all_data = []
    for i in range (4):
        url = "https://graph.facebook.com/{}/{}/insights?\
                metric={}&\
                period=day&\
                since={}&until={}&\
                access_token={}"\
                .format(fb_api_vers, page_id, metric_name, last_3_month, current_date, long_access_token)
        data = requests.get(url).json()
        data_clean = data['data'][0]['values']
        all_data.extend(data_clean)

        current_date = last_3_month - one_day_to_second
        last_3_month = current_date - three_month_to_second

    df = pd.DataFrame(data=all_data)
    # Clean datetime
    df['end_time'] = pd.to_datetime(df['end_time']).dt.date

    # Change cols' name
    cols_name = [metric_name, 'date']
    df.columns = cols_name

    # Reorder cols
    reorder_cols = ['date', metric_name]

    # Sort date
    df = df.sort_values(by=['date'])

    df = df[reorder_cols]
    
    return df

In [28]:
metric_name = 'page_fans'
page_fans = extract_metric_data(metric_name)
page_fans.columns = ['date', 'page fans']

total_fan = int(page_fans[-1:]['page fans'])

metric_name = 'page_views_total'
page_views = extract_metric_data(metric_name)
page_views.columns = ['date', 'page views']

metric_name = 'page_post_engagements'
page_engagements = extract_metric_data(metric_name)
page_engagements.columns = ['date', 'page engagements']

metric_name = 'page_video_views'
page_video_views = extract_metric_data(metric_name)
page_video_views.columns = ['date', 'page video views']

metric_name = 'page_posts_impressions_unique'
page_people_reach = extract_metric_data(metric_name)
page_people_reach.columns = ['date', 'page people reached']

df = page_fans
df = df.merge(page_views, left_on='date', right_on='date')
df = df.merge(page_engagements, left_on='date', right_on='date')
df = df.merge(page_video_views, left_on='date', right_on='date')
df = df.merge(page_people_reach, left_on='date', right_on='date')

### 1.1.2 Gender & Age

In [6]:
def extract_specific_metric_data(metric_name):

    url = "https://graph.facebook.com/{}/{}/insights?\
            metric={}&\
            period=day&\
            since={}&until={}&\
            access_token={}"\
            .format(fb_api_vers, page_id, metric_name, current_date, current_date, long_access_token)

    data = requests.get(url).json()
    data_clean = data['data'][0]['values'][0]['value']
    
    # Put data into a dataframe
    df = pd.DataFrame(data=data_clean.items())
    
    return df

In [7]:
# extract data
metric_name = 'page_fans_gender_age'
page_gender_age = extract_specific_metric_data(metric_name)

# rename cols' name
page_gender_age.columns = ['gender age', 'value']
page_gender_age[['gender', 'group age']] = page_gender_age['gender age'].str.split('.', expand=True)

# remove undefined user's gender
page_gender_age = page_gender_age[page_gender_age['gender'] != 'U']

# calculate the total number of users
total_m_f = sum(page_gender_age['value'])
# calculate the percentage of gender and age
page_gender_age['percentage'] = round(page_gender_age['value']/total_m_f*100, 1)

# extract targeted cols
cols = ['gender', 'group age', 'value', 'percentage']
page_gender_age = page_gender_age[cols]

### 1.1.2 City & Country

In [8]:
### CITY
# extract data
metric_name = 'page_fans_city'
page_city = extract_specific_metric_data(metric_name)

# rename cols' name
page_city.columns = ['city', 'value']
page_city['percentage'] = round(page_city['value'] / total_fan * 100, 2)

# top 15 cities with most fans
page_city = page_city[-15:]

In [9]:
### COUNTRY
def country_flag(country_code):
    return pycountry.countries.get(alpha_2=country_code).name

In [10]:
# extract data
metric_name = 'page_fans_country'
page_country = extract_specific_metric_data(metric_name)

# rename cols' name
page_country.columns = ['country', 'value']

# calculate percentage
page_country['percentage'] = round(page_country['value'] / total_fan * 100, 2)

# Clean country
# https://stackoverflow.com/questions/53923433/how-to-get-country-name-from-country-abbreviation-in-python-with-mix-of-alpha-2
list_alpha_2 = [i.alpha_2 for i in list(pycountry.countries)]
page_country['country'] = page_country['country'].apply(country_flag)

# sort cities based on the number of fans
page_country = page_country.sort_values(by=['value'], ascending=True)[-15:]

### Save File

In [11]:
def save_file(df_dict):
    # Check if a report file exist
    report_name = "report.xlsx"
    excel_file = Path(report_name)
    if excel_file.is_file() == False:
        writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
        writer.save()

    # Open Excel file
    book = load_workbook(report_name)
    writer = pd.ExcelWriter(report_name, engine='openpyxl') 
    writer.book = book

    # Store all existing sheets
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    # Add or Overwriting data to particualr Sheet
    for key, value in df_dict.items(): 
        value.to_excel(writer, key, index=False)
    
    writer.save()

In [12]:
df_dict = {}
df_dict['fb page performance'] = df
df_dict['fb gender age'] = page_gender_age
df_dict['fb city'] = page_city
df_dict['fb country'] = page_country

save_file(df_dict)

## 1.2 Post Performance

### DATA COLLECTION

In [13]:
number_post_retrieved_api = 25

all_posts = {'id': [],
             'date': [],
             
             'total_reach': [],
             'total_impression': [],
             'total_reaction': [],
             'comment': [],
             'share': [],
             'post_click': [],
             'engagement': [],
             
             'like': [],
             'love': [],
             'wow': [],
             'haha': [],
             'sad': [],
             'anger': [],
             
             'status_type': [], 
             'message': [],
             'url': []}

In [14]:
def extract_post_insight(posts, all_posts):
    for p in posts['data']:
        post_id = p['id']
        post_created_time = p['created_time']

        # print(post_id)
        
        url = "https://graph.facebook.com/{}/{}/insights?\
                metric=post_activity_by_action_type_unique, post_impressions, post_impressions_unique, post_engaged_users,\
                        post_reactions_like_total, post_reactions_love_total, post_reactions_wow_total,\
                        post_reactions_haha_total, post_reactions_sorry_total,post_reactions_anger_total&\
                access_token={}"\
                .format(fb_api_vers, post_id, long_access_token)
        post_insight = requests.get(url).json()

        # Like/Reaction
        try:
            total_reaction = post_insight['data'][0]['values'][0]['value']['like']
        except:
            total_reaction = 0
        
        # Comment
        try:
            comment = post_insight['data'][0]['values'][0]['value']['comment']
        except:
            comment = 0

        # Share
        try:
            share = post_insight['data'][0]['values'][0]['value']['share']
        except:
            share = 0

        # Total Impression
        try:
            total_impression = post_insight['data'][1]['values'][0]['value']
        except:
            total_impression = 0

        # Total Reach == Total Unique Impression
        try:
            total_reach = post_insight['data'][2]['values'][0]['value']
        except:
            total_reach = 0

        # Post Click
        try:
            post_click = post_insight['data'][3]['values'][0]['value']
        except:
            post_click = 0

        engagement = total_reaction + comment + post_click
        
        # Like
        try:
            like = post_insight['data'][4]['values'][0]['value']
        except:
            like = 0
        # Love
        try:
            love = post_insight['data'][5]['values'][0]['value']
        except:
            love = 0
        # Wow
        try:
            wow = post_insight['data'][6]['values'][0]['value']
        except:
            wow = 0
        # Haha
        try:
            haha = post_insight['data'][7]['values'][0]['value']
        except:
            haha = 0
        # Sad
        try:
            sad = post_insight['data'][8]['values'][0]['value']
        except:
            sad = 0
        # Anger
        try:
            anger = post_insight['data'][9]['values'][0]['value']
        except:
            anger = 0
        
        
        url = "https://graph.facebook.com/{}/{}?\
                fields=status_type, message, permalink_url&\
                access_token={}"\
                .format(fb_api_vers, post_id, long_access_token)
        post_insight = requests.get(url).json()
        
        # status_type: photo, video
        status_type = post_insight['status_type']
        status_type = status_type[status_type.find("_")+1 :]
        
        # message
        try:
            message = post_insight['message']
        except:
            message = ''
            
        # post url
        url = post_insight['permalink_url']
        
        
        all_posts['id'].append(post_id)
        all_posts['date'].append(post_created_time)
        
        all_posts['total_reaction'].append(total_reaction)
        all_posts['comment'].append(comment)
        all_posts['share'].append(share)
        all_posts['total_impression'].append(total_impression)
        all_posts['total_reach'].append(total_reach)
        all_posts['post_click'].append(post_click)
        all_posts['engagement'].append(engagement)
        
        all_posts['like'].append(like)
        all_posts['love'].append(love)
        all_posts['wow'].append(wow)
        all_posts['haha'].append(haha)
        all_posts['sad'].append(sad)
        all_posts['anger'].append(anger)
        
        all_posts['status_type'].append(status_type)
        all_posts['message'].append(message)
        all_posts['url'].append(url)

In [15]:
def post_number(number_of_posts, all_posts):
    number_api_calling = math.ceil(number_of_posts/number_post_retrieved_api)
    print('number_api_calling', number_api_calling)
    
    for i in range (number_api_calling):
        if i == 0:
            # 1st 25 posts
            url = "https://graph.facebook.com/{}/{}?\
                    fields=posts&\
                    access_token={}"\
                    .format(fb_api_vers, page_id, long_access_token)
            posts = requests.get(url).json()
            posts = posts['posts']
        
        else:
            try:
                url = posts['paging']['next']
                posts = requests.get(url).json()
            except:
                print('The number of posts on page is less than is required')
                break
        
        if i == number_api_calling - 1:
            remaining_posts = number_of_posts - number_post_retrieved_api*i
            posts['data'] = posts['data'][:remaining_posts]
        

        extract_post_insight(posts, all_posts)     

In [16]:
post_number(250, all_posts)

number_api_calling 10


### DATA CLEANING

In [17]:
def display_outlier(df):
    print(("impressions: {0:<5} engagement: {1:<5} {2:<10}".format(df['total_impression'], df['engagement'], df['url'])))

In [18]:
def clean_datetime(datetime):
    month = datetime.month
    day = datetime.day

    if month > 3 and month < 11:
        datetime = datetime + pd.to_timedelta(-7,unit='hour')
    elif month == 3 and day > 14:
        datetime = datetime + pd.to_timedelta(-7,unit='hour')
    elif month == 11 and day < 7:
        datetime = datetime + pd.to_timedelta(-7,unit='hour')
    else:
        datetime = datetime + pd.to_timedelta(-8,unit='hour')
    
    return datetime

In [19]:
def remove_timezone(datetime):
    return datetime.strftime('%Y-%m-%d %H:%M:%S')

In [20]:
df = pd.DataFrame(data = all_posts)
# dropping ALL duplicte values
df.drop_duplicates(subset ="id",
                   keep = 'first', inplace = True)

In [21]:
# Change datatype of date
df['date'] = pd.to_datetime(df['date'])

# Just apply for Pacific time zone
df['date'] = df['date'].apply(clean_datetime)

# Remove timezone
df['date'] = df['date'].apply(remove_timezone)

### Top 20 post with most Reach, Impression, Engagement

In [22]:
columns_display = ['url','date','total_reach', 'total_impression',
                   'total_reaction','comment', 'share', 'post_click', 'engagement',
                   'status_type']

top_20_reach = df.sort_values('total_reach', ascending=False)[:20][columns_display]
top_20_impression = df.sort_values('total_impression', ascending=False)[:20][columns_display]
top_20_engagement = df.sort_values('engagement', ascending=False)[:20][columns_display]

### Save File

In [23]:
df_dict = {}
df_dict['fb posts'] = df
df_dict['fb top 20 reached posts'] = top_20_reach
df_dict['fb top 20 impressioned posts'] = top_20_impression
df_dict['fb top 20 engageged posts'] = top_20_engagement

save_file(df_dict)

# 2. INSTAGRAM

## 2.0 Authentication

In [32]:
def insta_id_extraction():
    url = "https://graph.facebook.com/v10.0/{}?\
            fields=instagram_business_account&\
            access_token={}"\
            .format(page_id,long_access_token)
    data = requests.get(url).json()
    
    return data['instagram_business_account']['id']

In [33]:
fb_api_vers = authen['fb_api_vers']
long_access_token = authen['ig_token']
page_id = authen['fb_page_id']
insta_id = insta_id_extraction()

## 2.1 Page / Account Performance

### 2.1.1 Reach, Profile View

In [35]:
def ig_extract_metric_data(metric_name):
    now = datetime.now()
    current_date = math.floor(datetime.timestamp(now))
    one_day_to_second = 60*60*24
    one_month_to_second = 60*60*24*30
    last_1_month = current_date - one_month_to_second

    all_data = []
    for i in range (12):
        url = "https://graph.facebook.com/{}/{}/insights?\
                metric={}&\
                period=day&\
                since={}&until={}&\
                access_token={}"\
                .format(fb_api_vers, insta_id, metric_name, last_1_month, current_date, long_access_token)

        data = requests.get(url).json()
        data_clean = data['data'][0]['values']
        all_data.extend(data_clean)

        current_date = last_1_month
        last_1_month = current_date - one_month_to_second

    df = pd.DataFrame(data=all_data)

    # Clean datetime
    df['end_time'] = pd.to_datetime(df['end_time']).dt.date

    # Change cols' name
    cols_name = [metric_name, 'date']
    df.columns = cols_name

    # Reorder cols
    reorder_cols = ['date', metric_name]
    df = df[reorder_cols]
    
    # Sort date
    df = df.sort_values(by=['date'])

    return df

In [36]:
metric_name = 'reach'
ig_page_people_reach = ig_extract_metric_data(metric_name)

metric_name = 'profile_views'
ig_profile_views = ig_extract_metric_data(metric_name)

ig_df = ig_page_people_reach
ig_df = ig_df.merge(ig_profile_views, left_on='date', right_on='date')

### 2.1.2 Gender & Age

In [37]:
def ig_extract_specific_metric_data(metric_name):
    url = "https://graph.facebook.com/{}/{}/insights?\
        metric={}&\
        period=lifetime&\
        access_token={}"\
        .format(fb_api_vers, insta_id, metric_name, long_access_token)
    
    data = requests.get(url).json()
    data_clean = data['data'][0]['values'][0]['value']
    
    # Put data into a dataframe
    df = pd.DataFrame(data=data_clean.items())
    
    return df

In [38]:
metric_name = 'audience_gender_age'
ig_gender_age = ig_extract_specific_metric_data(metric_name)

# rename cols' name
ig_gender_age.columns = ['gender age', 'value']
ig_gender_age[['gender', 'group age']] = ig_gender_age['gender age'].str.split('.', expand=True)

# remove undefined user's gender
ig_gender_age = ig_gender_age[ig_gender_age['gender'] != 'U']

# calculate the total number of users
total_m_f = sum(ig_gender_age['value'])
# calculate the percentage of gender and age
ig_gender_age['percentage'] = round(ig_gender_age['value']/total_m_f*100, 1)

# extract targeted cols
cols = ['gender', 'group age', 'value', 'percentage']
ig_gender_age = ig_gender_age[cols]

### 2.1.3 City & Country

In [39]:
### CITY

metric_name = 'audience_city'
ig_city = ig_extract_specific_metric_data(metric_name)

# rename cols' name
ig_city.columns = ['city', 'value']
ig_city['percentage'] = round(ig_city['value'] / total_m_f * 100, 2)

In [40]:
### COUNTRY
def country_flag(country_code):
    return pycountry.countries.get(alpha_2=country_code).name

In [41]:
metric_name = 'audience_country'
ig_country = ig_extract_specific_metric_data(metric_name)

# rename cols' name
ig_country.columns = ['country', 'value']

# calculate percentage
ig_country['percentage'] = round(ig_country['value'] / total_m_f * 100, 2)

# Clean country
# https://stackoverflow.com/questions/53923433/how-to-get-country-name-from-country-abbreviation-in-python-with-mix-of-alpha-2
list_alpha_2 = [i.alpha_2 for i in list(pycountry.countries)]
ig_country['country'] = ig_country['country'].apply(country_flag)

# sort cities based on the number of fans
ig_country = ig_country.sort_values(by=['value'], ascending=True)[-15:]

### 2.1.4 Online Followers

In [42]:
current_date = math.floor(datetime.timestamp(now))
two_day = current_date - 60*60*24*3
three_day = current_date - 60*60*24*4

url = "https://graph.facebook.com/{}/{}/insights?\
        metric=online_followers&\
        period=lifetime&\
        since={}&until={}&\
        access_token={}"\
        .format(fb_api_vers, insta_id, three_day, two_day, long_access_token)

ig_online_followers = requests.get(url).json()

ig_online_followers = ig_online_followers['data'][0]['values'][0]['value']
# Put data into a dataframe
ig_online_followers = pd.DataFrame(data=ig_online_followers.items())
ig_online_followers.columns = ['hour', 'follower_online']

### Save File

In [43]:
df_dict = {}
df_dict['ig account performance'] = ig_df
df_dict['ig gender age'] = ig_gender_age
df_dict['ig city'] = ig_city
df_dict['ig country'] = ig_country
df_dict['ig online follower'] = ig_online_followers

save_file(df_dict)

## 2.2 Post Performance

### DATA COLLECTION

In [44]:
def api_impression_reach(post_id):
    url = 'https://graph.facebook.com/{}/{}/insights?\
            metric=impressions,reach&\
            access_token={}'\
            .format(fb_api_vers, post_id, long_access_token)
    return requests.get(url).json()

def api_like_comment_createdTime_mediaType_link(post_id):
    url = 'https://graph.facebook.com/{}/{}?\
            fields=like_count,comments_count,media_type,media_url,permalink,timestamp,caption&\
            access_token={}'\
            .format(fb_api_vers, post_id, long_access_token)
    return requests.get(url).json()

In [45]:
number_posts = 75
number_post_retrieved_api = 25

all_posts = {'id': [],
             'date': [],
             
             'total_reach': [],
             'total_impression': [],
             'like/reaction': [],
             'comment': [],
             'engagement': [],
             
             'media_type': [],
             'url': [],
             'caption': []}

In [46]:
def extract_post_insight(posts, all_posts):    
    for p in posts['data']:
        post_id = p['id']

        post = api_impression_reach(post_id)

        total_impression = post['data'][0]['values'][0]['value']
        total_reach = post['data'][1]['values'][0]['value']

        post = api_like_comment_createdTime_mediaType_link(post_id)

        like_reaction = post['like_count']
        comment = post['comments_count']
        engagement = like_reaction + comment
        created_time = post['timestamp']
        media_type = post['media_type']
        url = post['permalink']
        caption = post['caption']

        all_posts['id'].append(post_id)
        all_posts['date'].append(created_time)
        all_posts['total_reach'].append(total_reach)
        all_posts['total_impression'].append(total_impression)
        all_posts['like/reaction'].append(like_reaction)
        all_posts['comment'].append(comment)
        all_posts['engagement'].append(engagement)
        all_posts['media_type'].append(media_type)
        all_posts['url'].append(url)
        all_posts['caption'].append(caption)

In [47]:
def post_number(number_of_posts, all_posts):
    number_api_calling = math.ceil(number_of_posts/number_post_retrieved_api)
    print('number_api_calling', number_api_calling)
    
    for i in range (number_api_calling):
        if i == 0:
            # 1st 25 posts
            url =  "https://graph.facebook.com/{}/{}/media?access_token={}"\
                    .format(fb_api_vers, insta_id, long_access_token)
        else:
            url = posts['paging']['next']
        
        posts = requests.get(url).json()
        
        if i == number_api_calling - 1:
            remaining_posts = number_of_posts - number_post_retrieved_api*i
            posts['data'] = posts['data'][:remaining_posts]
            
        extract_post_insight(posts, all_posts)

In [48]:
post_number(250, all_posts)

number_api_calling 10


### DATA CLEANING

In [49]:
df = pd.DataFrame(data = all_posts)
# dropping ALL duplicte values
df.drop_duplicates(subset ="id",
                   keep = 'first', inplace = True)

In [51]:
# Change datatype of date
df['date'] = pd.to_datetime(df['date'])

# Just apply for Pacific time zone
df['date'] = df['date'].apply(clean_datetime)

# Remove timezone
df['date'] = df['date'].apply(remove_timezone)

### Top 20 post with most Reach, Impression, Engagement

In [52]:
columns_display = ['url','date','total_reach', 'total_impression',
                   'like/reaction','comment', 'engagement',
                   'media_type']

top_20_reach = df.sort_values('total_reach', ascending=False)[:20][columns_display]
top_20_impression = df.sort_values('total_impression', ascending=False)[:20][columns_display]
top_20_engagement = df.sort_values('engagement', ascending=False)[:20][columns_display]

### Save File

In [53]:
df_dict = {}
df_dict['ig posts'] = df
df_dict['ig top 20 reached posts'] = top_20_reach
df_dict['ig top 20 impressioned posts'] = top_20_impression
df_dict['ig top 20 engageged posts'] = top_20_engagement

save_file(df_dict)